In [0]:
# Cell 0: Generic Definition Hash Generator
#
# Purpose: Reusable notebook to compute definition_hash for any metadata table
#
# This notebook enables creating a "clinical metadata library" by computing
# deterministic hashes for metadata definitions. Same definition across different
# trials/vendors/streams produces the same hash, enabling gold table deduplication.

import json
from clinical_data_standards_framework.utils import compute_definition_hash_spark
from pyspark.sql import functions as F

print("Generic Definition Hash Generator")
print("="*80)

In [0]:
# Cell 1: Read Configuration from Setup Task

# Get configuration from setup task (like other notebooks)
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")

# Extract catalog and schema
catalog = globals_dict['catalog']
silver_schema = globals_dict['silver_schema']

# Get job parameters
table_name = dbutils.widgets.get("table_name")  # Just table name, not full path
hash_fields_json = dbutils.widgets.get("hash_fields_json")
pk_columns_json = dbutils.widgets.get("pk_columns_json")

# Parse JSON parameters
hash_fields = json.loads(hash_fields_json)
pk_columns = json.loads(pk_columns_json)

# Build full table name using catalog/schema from config
table_full_name = f"{catalog}.{silver_schema}.{table_name}"

print(f"Catalog: {catalog}")
print(f"Schema: {silver_schema}")
print(f"Table: {table_full_name}")
print(f"Hash fields ({len(hash_fields)}): {hash_fields}")
print(f"PK columns ({len(pk_columns)}): {pk_columns}")
print(f"Principal: {created_by_principal}")
print("="*80)


In [0]:
# Cell 2: Compute Definition Hash

print("\nReading table...")
df = spark.table(table_full_name)
total_rows = df.count()
print(f"Total rows: {total_rows:,}")

# Convert codelist_values array to sorted string for hashing
# This ensures same values = same hash, regardless of order
if "codelist_values" in df.columns:
    print("\nPreparing codelist values for hashing...")
    df = df.withColumn(
        "codelist_values_str",
        F.when(
            F.col("codelist_values").isNotNull(),
            F.array_join(
                F.array_sort(
                    F.transform(F.col("codelist_values"), lambda x: F.lower(x))
                ), 
                "|"
            )
        ).otherwise(F.lit(""))  # NULL → empty string
    )
    # Replace "codelist_values" with "codelist_values_str" in hash_fields
    hash_fields_for_hashing = []
    for field in hash_fields:
        if field == "codelist_values":
            hash_fields_for_hashing.append("codelist_values_str")
        else:
            hash_fields_for_hashing.append(field)
else:
    hash_fields_for_hashing = hash_fields

print("\nComputing definition hash...")
df_with_hash = compute_definition_hash_spark(
    df, 
    hash_fields=hash_fields_for_hashing,
    output_column="definition_hash"
)

print("\nSample rows with hash:")
sample_cols = pk_columns[:2] + ['definition_hash']
df_with_hash.select(sample_cols).limit(5).show(truncate=False)

unique_hashes = df_with_hash.select("definition_hash").distinct().count()
print(f"\nUnique definition hashes: {unique_hashes:,}")
print(f"Deduplication ratio: {unique_hashes / total_rows:.1%}")
print("="*80)

In [0]:
# Cell 3: Merge Hash Back to Table

print("\nUpdating table with definition hash...")

df_with_hash.createOrReplaceTempView("temp_hash_view")

merge_conditions = " AND ".join([
    f"target.{col} = source.{col}" for col in pk_columns
])

merge_sql = f"""
MERGE INTO {table_full_name} AS target
USING temp_hash_view AS source
ON {merge_conditions}
WHEN MATCHED THEN UPDATE SET
  target.definition_hash = source.definition_hash,
  target.last_updated_by_principal = '{created_by_principal}',
  target.last_updated_ts = current_timestamp()
"""

spark.sql(merge_sql)

updated_count = spark.sql(f"""
  SELECT COUNT(*) as cnt
  FROM {table_full_name}
  WHERE definition_hash IS NOT NULL
""").first()['cnt']

print(f"✓ Updated {updated_count:,} rows with definition hash")
print("="*80)

print("\nDefinition hash distribution (top 10 most common):")
spark.sql(f"""
  SELECT 
    definition_hash,
    COUNT(*) as usage_count
  FROM {table_full_name}
  WHERE definition_hash IS NOT NULL
  GROUP BY definition_hash
  ORDER BY usage_count DESC
  LIMIT 10
""").show(truncate=False)

print("\n✓ Definition hash computation complete!")